# Rappels jour 2 

* gotchas sur les fonctions 
    * pas d'arguments mutables en entré
    * accès aux varaible : L(ocal) E(nglobing) G(lobal) B(uiltin)
    * les variables englobing sont passées par closure
    * en python les closures : 
        * sont late binding 
        * sont en lecture seule
* curryfication =>  passer d'une fonction à N arguments à N fonctions imbriquées à 1 argument chacune 
* décorateurs
    * on peut le comprendre comme une composée de fonction `toto = deco(toto)` 
    * on peut chainer les décorateurs, l'ordre a une importance 
    * la fonction a l'intérieur s'appelle `wrapper` par convention 
    * dans le cas général, la bonne pratique consiste à décorer le wrapper avec `functools.wraps` pour conserver les métadonnées de la fonction d'origine (nom, docstring)
* les décorateurs peuvent être paramétrés, on rajoute une fonction qui va générer le décorateur qui nous intéresse

In [9]:
class Toto:
    def __init__(self, a):
        self.a = a 
    
    def hello(self):
        print(self.a)
    
    def __call__(self, b):
        print(b)
        
        
t = Toto("a")
t("b")

b
None


In [2]:
import inspect

In [3]:
inspect.getsourcefile(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [4]:
inspect.getsourcelines(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [23]:
from typing import Callable, TypeVar

T = TypeVar('T')

class MyDecorator:
    my_list = []
    
    def __init__(self, func: Callable[[T], T]):
        self.func = func
        self.my_list.append(func)
        
    def __call__(self, *args, **kwargs) -> T:
        print("Before function call")
        result = self.func(*args, **kwargs)
        print("After function call")
        return result
    
    def analyse(self):
        self.my_list

In [24]:
@MyDecorator
def my_function(arg: str) -> str:
    return arg.upper()


@MyDecorator
def toto(arg: str) -> str:
    return arg.upper()

my_function("hello")

Before function call
After function call


'HELLO'

In [25]:
MyDecorator.my_list

[<function __main__.my_function(arg: str) -> str>,
 <function __main__.toto(arg: str) -> str>]

# Gestion de fichiers / gestionnaires de contexte

In [ ]:
f = open